# dev measure birth/death events recursively

In [5]:
from lib.my_initialization import *
from lib.controller.controller_LR import get_one_step_explicit_synchronous_splitting
from lib.model.LR_model import *
from lib.utils.utils_traj import *
from lib.routines.bdrates import *
from lib.measure.utils_measure_tips_cpu import *
from lib.utils.stack_txt_LR import *
from lib.viewer import *
import trackpy
from lib.utils import load_buffer

#automate the boring stuff
# from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
    
%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## TODO: dev python routine that takes in a spiral tip in pdict and follows it iteratively to its death, zooming in as needed, until a certain absolute tolerance is found in a zero minimum range is reached.  
Consider dict_atol={'batch':1e-4,'test':1e-2,'precise':1e-6}



input:dict_topo,point,width,height
output:pid_lst within r distance of point

In [6]:
def get_particles_in_range(dict_topo,point,width,height,r):
    
    dict_topo['xy']
    
    return pid_lst_within_r

In [16]:
txt_fn=f'{nb_dir}/Data/test_data/ic008.33_t_218.8.npz'
t=218.8
txt=load_buffer(txt_fn)
inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
width,height=txt.shape[:2]
print(txt.shape)

(200, 200, 18)


In [17]:
# get_one_step
dt, arr39, one_step = get_one_step_explicit_synchronous_splitting(
    nb_dir,
    dt,
    width,
    height,
    ds=5.0,
    diffCoef=0.0005,
    Cm=1.0)
#the heavyweight spiral tip measures
V_threshold=-50
jump_threshold = 40
ds=5.0
comp_dict_tips=get_comp_dict_topo_full_color(width=width,height=height,level1=V_threshold,level2=0,
                                             jump_threshold=jump_threshold,
                                            ds=ds)
# comp_dict_tips=get_compute_all_spiral_tips(width, height, mode='simp')

In [15]:
#measure tips that are already present
img=inVc[...,0];dimgdt=dVcdt[...,0]
# dict_topo=comp_dict_topo_simple(img,dimgdt,t)
dict_topo=comp_dict_topo_full_color(img,dimgdt,t,txt)
pid_lst_all=sorted(dict_topo.keys())
n_tips=len(pid_lst_all)
n_tips

22

In [ ]:
1